# **Problem Set 9**

## **Tasks**
- Assembling A and S matrices
- Calculating d
- Grouping impacts of sub-components into the components
- Answering multiple choice questions

## **Passing requirements**
- **At least 80 points**
- **Question 1) has to be delivered on Blackboard in Assignments -> PS9 -> "PS9 - Questions BB"**
- **Question 5) has to be delivered on Blackboard in Assignments -> PS9 -> "PS9 - Questions BB"**

**Answers to questions 1) and 5) count for 20 points in total. Write your answers in the word template provided in Assignments -> PS9 -> "PS9 - Questions BB" converted to a pdf file!**

**_Please run the cell below before you start the assignment_**

In [1]:
# Import required Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import files_PS9_V1_0.run_SPA_folder.run_SPA as run_SPA

# Folder for the files
files_folder = "files_PS9_V1_0/"

### In case formulas are entered in the csv files & correcting the datatypes
def convert_formulas(matrix):
    for row in matrix.index:
        for column in matrix.columns:
            if type(matrix.loc[row, column]) == str:
                if " " in matrix.loc[row, column]:
                    matrix.loc[row, column] = matrix.loc[row, column].lstrip(' ')
                if "=" in matrix.loc[row, column]:
                    matrix.loc[row, column] = matrix.loc[row, column].lstrip('=')
                matrix.loc[row, column] = pd.eval(matrix.loc[row, column])
    ### Get the data types right in the DataFrames
    s = matrix.select_dtypes(include='object').columns
    matrix[s] = matrix[s].astype("float64")
    return matrix

## **LCA of a wind turbine**

A wind turbine consists of three main components: the rotor, the nacelle and the tower. Each of the main components has several sub-components, listed in Table 1 and in **"Foreground_Background_list_PS9.csv"** in **files_PS9_V1_0**. Table 1 presents the number of sub-components needed per component. The table in **"Foreground_Background_list_PS9.csv"** presents the material composition of each sub-component and energy requirements for the assembly of the turbine. It is assumed there are no direct emissions in this system.

|Table 1: Components and sub-components of the wind turbine|||
|-|-|-|
|**Component**|**Sub-component**|**Number of sub-components needed per component**|
|Rotor|Blade|3|
||Hub w/nose cone|1|
|Nacelle|Generator|1|
||Gearbox|1|
||Cover / housing|1|
||Main frame|1|
||Main shaft|1|
||Transformer|1|
|Tower|Tubular steel|1|
||Internals|1|

_**Disclaimer**: Mainstream LCA assessments normally use commercial LCI databases, such as Ecoinvent, for their background systems. These databases are, as the name indicates, commercial in nature and are as such not open access. We are therefore using an open-access mixed-unit version of EXIOBASE. This is an Input-Output database (IOT). So for your applications, you will in principle be using an IO-Hybrid LCA model rather than a conventional process-based LCA model. Have a look at [DOI: 10.5281/zenodo.7244918] and [DIO: 10.5281/zenodo.3583070] if you are interested in more details about the dataset you will be using. Please though note, that this dataset is only intended for training purposes in this course, so if you are planning to do project or thesis work on LCA, consult your supervisor to get access to suitable databases for your purpose._

## **1. Flowchart** _(10 points)_

**Instructions:** Draw a flowchart of your system. Label all the nodes and draw the arrows for all the flows. Label the flows from the foreground system (Aff part of the flowchart only) with their values and units.

This question brings 10 points and is **mandatory** for passing the assignment.

_The flowchart needs to be readable for us: you can do the flowchart by hand (drawing on paper and taking a picture/scanning) or you can use a software (draw on a tablet, use PowerPoint, Lucidchart, Miro, etc.). **Place the flowchart in the word template saved in pdf!**_

## **2. Data preparation** _(2.5 points)_

Run the cell below to import `Aff`, `Abf`, `Afb` and `Abb` as Pandas DataFrames.

In [2]:
Aff = convert_formulas(pd.read_csv(files_folder + 'Aff.csv', sep = ',', index_col = [0]))
Abf = convert_formulas(pd.read_csv(files_folder + 'Abf.csv', sep = ',', index_col = [0]))
Afb = convert_formulas(pd.read_csv(files_folder + 'Afb.csv', sep = ',', index_col = [0]))
Abb = convert_formulas(pd.read_csv(files_folder + 'Abb.csv', sep = ',', index_col = [0]))

### List of background processes for the checking etc.
list_backgroundprocesses = list(Abb.index)

Run the cell below to get the `C` matrix.

In [3]:
C = convert_formulas(pd.read_csv(files_folder + 'C.csv', sep = ',', index_col = [0]))

Run the cell below to get the `Sb` matrix.

In [4]:
Sb = convert_formulas(pd.read_csv(files_folder + 'Sb.csv', sep = ',', index_col = [0]))

Define `Sf` (think if it should contain values or not) and build the entire `S` matrix.

In [5]:
Sf = pd.DataFrame(index=Sb.index, columns=['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower', 'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing', 'Main frame', 'Main shaft', 'Transformer', 'Tubular steel', 'Internals'])
Sf = Sf.fillna(0)
#display(Sf)
S = pd.concat([Sf, Sb], axis = 1)
S = S.fillna(0)
#display(S)

**_Run the cell under to check the format of your answer(s)_**

In [6]:
assert S.isnull().values.any() == False, 'There are still NaN values in the matrix'
assert (S.index == Sb.index).all(), 'The index do not correspond'
assert (S.columns == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower', 'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing', 'Main frame', 'Main shaft', 'Transformer', 'Tubular steel', 'Internals',
                      ] + list_backgroundprocesses).all(), 'The columns do not correspond'

## **3. Total impacts of the wind turbine** _(45 points)_

#### **3a)** _(10 points)_ Fill in `Aff`
For that, you have two options: 
- Option 1: filling in csv file located in the **files_PS9_V1_0** folder and save the csv files by using "ctrl+s". **If you are calculating some numbers, you should enter the formulas in the table.**
- Option 2: filling in the table in the cell below using the pandas loc fonction

In [7]:
Aff = Aff.fillna(0)
display(Aff)

,Wind turbine,Turbine assembly,Rotor,Nacelle,Tower,Blade,Hub w/nose cone,Generator,Gearbox,Cover / housing,Main frame,Main shaft,Transformer,Tubular steel,Internals
Wind turbine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Turbine assembly,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rotor,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nacelle,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tower,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blade,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hub w/nose cone,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Generator,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gearbox,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cover / housing,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**_Run the cell under to check the format of your answer(s)_**

In [8]:
assert Aff.isnull().values.any() == False, 'There are still NaN values in the matrix'
assert (Aff.index == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower',
       'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing',
       'Main frame', 'Main shaft', 'Transformer', 'Tubular steel',
       'Internals'
                     ]).all(), 'The index do not correspond'
assert (Aff.columns == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower',
       'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing',
       'Main frame', 'Main shaft', 'Transformer', 'Tubular steel',
       'Internals'
                       ]).all(), 'The columns do not correspond'

#### **3b)** _(10 points)_ Fill in `Abf`

Use the file Foreground_Background_list_PS9.csv to fill in `Abf`. For that, you have two options: 
- Option 1: filling in csv file located in the **files_PS9_V1_0** folder and save the csv files by using "ctrl+s". **If you are calculating some numbers, you should enter the formulas in the table.**
- Option 2: filling in the table in the cell below using the function `fill_in_matrix_from_table(matrix, table)` which is given to transform data from a one column table to a matrix. 

Using Option 2 function avoids to filling in the matrix manually and prevents forgetting values, but you are free to choose. 

In [9]:
def fill_in_Abf_from_table(Abf, table):
    for i in Abf.index:
        for j in Abf.columns:
            if table[(table['Background process'] == i) & (table['Foreground process'] == j)]['Value'].empty == False:
                Abf.loc[i, j] = table[(table['Background process'] == i) & (table['Foreground process'] == j)]['Value'].values[0]
    return Abf

### This is importing the table for using the function fill_in_matrix_from_table(matrix, table)
Abf_list = pd.read_csv(files_folder + 'Foreground_Background_list_PS9.csv')

In [10]:
fill_in_Abf_from_table(Abf, Abf_list)
Abf = Abf.fillna(0)
display(Abf)

,Wind turbine,Turbine assembly,Rotor,Nacelle,Tower,Blade,Hub w/nose cone,Generator,Gearbox,Cover / housing,Main frame,Main shaft,Transformer,Tubular steel,Internals
Paddy rice - tonnes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wheat - tonnes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cereal grains nec - tonnes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vegetables; fruit; nuts - tonnes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oil seeds - tonnes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Membership organisation services n.e.c. (91) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Recreational; cultural and sporting services (92) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Other services (93) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Private households with employed persons (95) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**_Run the cell under to check the format of your answer(s)_**

In [11]:
assert Abf.isnull().values.any() == False, 'There are still NaN values in the matrix'
assert (Abf.index == list_backgroundprocesses).all(), 'The index do not correspond'
assert (Abf.columns == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower',
       'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing',
       'Main frame', 'Main shaft', 'Transformer', 'Tubular steel',
       'Internals'
                       ]).all(), 'The columns do not correspond'

#### **3c)** _(10 points)_ Fill in `Afb`, build the `A` matrix and calculate the `L` matrix

In [12]:
Afb = Afb.fillna(0)
A1 = pd.concat([Aff, Afb], axis = 1)
A2 = pd.concat([Abf, Abb], axis = 1)
A = pd.concat([A1, A2], axis = 0)
A = A.fillna(0)
display(A)

I = np.identity(len(A))
Larr = np.linalg.inv(I-A)
L = pd.DataFrame(Larr, index = A.index, columns = A.columns)
display(L)

,Wind turbine,Turbine assembly,Rotor,Nacelle,Tower,Blade,Hub w/nose cone,Generator,Gearbox,Cover / housing,...,Paper for treatment: landfill - tonnes,Plastic waste for treatment: landfill - tonnes,Inert/metal/hazardous waste for treatment: landfill - tonnes,Textiles waste for treatment: landfill - tonnes,Wood waste for treatment: landfill - tonnes,Membership organisation services n.e.c. (91) - Meuro,Recreational; cultural and sporting services (92) - Meuro,Other services (93) - Meuro,Private households with employed persons (95) - Meuro,Extra-territorial organizations and bodies - Meuro
Wind turbine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
Turbine assembly,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
Rotor,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
Nacelle,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
Tower,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Membership organisation services n.e.c. (91) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.957063e-08,5.712072e-08,3.864001e-09,1.983862e-07,9.069771e-08,0.001323,0.000923,0.001011,0.000778,0.0
Recreational; cultural and sporting services (92) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.639514e-08,4.854480e-08,7.695309e-09,2.613004e-08,4.134645e-08,0.020345,0.062328,0.006751,0.000203,0.0
Other services (93) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.967347e-08,3.223699e-08,8.743078e-09,5.235152e-08,3.120477e-08,0.009806,0.005702,0.028257,0.000787,0.0
Private households with employed persons (95) - Meuro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0


,Wind turbine,Turbine assembly,Rotor,Nacelle,Tower,Blade,Hub w/nose cone,Generator,Gearbox,Cover / housing,...,Paper for treatment: landfill - tonnes,Plastic waste for treatment: landfill - tonnes,Inert/metal/hazardous waste for treatment: landfill - tonnes,Textiles waste for treatment: landfill - tonnes,Wood waste for treatment: landfill - tonnes,Membership organisation services n.e.c. (91) - Meuro,Recreational; cultural and sporting services (92) - Meuro,Other services (93) - Meuro,Private households with employed persons (95) - Meuro,Extra-territorial organizations and bodies - Meuro
Wind turbine,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
Turbine assembly,1.000000,1.000000,-1.688575e-16,1.134552e-15,-2.484938e-15,2.289693e-17,1.902753e-16,-7.877254e-16,1.382099e-16,-2.456835e-17,...,2.151672e-19,-9.629077e-19,3.690168e-17,-1.754444e-19,-5.041175e-20,1.872460e-15,2.317382e-15,-2.423406e-16,1.938725e-15,-0.0
Rotor,1.000000,1.000000,1.000000e+00,-1.511589e-16,-5.702544e-15,1.480297e-16,-3.284410e-16,-2.983706e-16,1.166962e-16,9.244300e-17,...,1.695540e-19,-9.847588e-19,3.690585e-17,-1.642594e-20,-3.439093e-20,-4.891776e-16,-1.217331e-15,5.600887e-16,2.485100e-16,-0.0
Nacelle,1.000000,1.000000,-3.560520e-16,1.000000e+00,-2.109424e-15,6.020033e-17,1.485357e-16,-9.488937e-16,2.264898e-16,-2.877202e-17,...,1.740388e-19,-9.968784e-19,3.688510e-17,5.301897e-20,-5.272780e-20,-2.019218e-15,-1.269818e-15,-3.330669e-15,-4.440892e-16,0.0
Tower,1.000000,1.000000,-3.560520e-16,4.410534e-16,1.000000e+00,6.020033e-17,1.485357e-16,-9.488937e-16,2.264898e-16,-2.877202e-17,...,1.740388e-19,-9.968784e-19,3.688510e-17,5.301897e-20,-5.272780e-20,-2.019218e-15,-1.269818e-15,-3.330669e-15,-4.440892e-16,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Membership organisation services n.e.c. (91) - Meuro,0.003433,0.003433,2.352439e-04,7.450691e-04,2.388557e-03,2.063834e-05,1.733289e-04,1.312302e-04,2.790986e-04,1.547876e-05,...,1.202673e-07,8.407082e-08,8.622270e-09,2.413141e-07,1.237998e-07,1.002190e+00,1.620988e-03,1.637203e-03,1.024316e-03,0.0
Recreational; cultural and sporting services (92) - Meuro,0.004948,0.004948,3.423952e-04,1.079615e-03,3.468194e-03,3.028354e-05,2.515446e-04,1.893901e-04,4.049894e-04,2.271266e-05,...,1.781869e-07,1.195437e-07,1.908108e-08,1.357534e-07,1.303451e-07,2.381856e-02,1.068182e+00,8.679555e-03,8.163546e-04,0.0
Other services (93) - Meuro,0.009906,0.009906,6.301092e-04,2.124423e-03,7.033889e-03,4.063761e-05,5.081964e-04,3.551187e-04,8.215495e-04,3.047821e-05,...,1.797887e-07,1.124352e-07,2.193005e-08,1.783914e-07,1.251108e-07,1.181916e-02,7.620371e-03,1.030263e+00,1.460727e-03,0.0
Private households with employed persons (95) - Meuro,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0


**_Run the cell under to check the format of your answer(s)_**

In [13]:
assert (L.index == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower', 'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing', 'Main frame', 'Main shaft', 'Transformer', 'Tubular steel', 'Internals', 
                    ] + list_backgroundprocesses).all(), 'The index do not correspond'
assert (L.columns == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower', 'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing', 'Main frame', 'Main shaft', 'Transformer', 'Tubular steel', 'Internals',
                      ] + list_backgroundprocesses).all(), 'The columns do not correspond'

#### **3d)** _(7.5 points)_ Calculate the output vector `x`

Run the cell below to get the `y` vector.

In [14]:
y = pd.read_csv(files_folder + 'y.csv', sep = ',', index_col = [0])

Fill in `y`, calculate `x` and rename its column into "Output".

In [15]:
y = y.fillna(0)
x = L @ y
x = x.rename(columns = {'Final demand' : 'Output'})
display(x)

,Output
Wind turbine,1.000000
Turbine assembly,1.000000
Rotor,1.000000
Nacelle,1.000000
Tower,1.000000
...,...
Membership organisation services n.e.c. (91) - Meuro,0.003433
Recreational; cultural and sporting services (92) - Meuro,0.004948
Other services (93) - Meuro,0.009906
Private households with employed persons (95) - Meuro,0.000000


**_Run the cell under to check the format of your answer(s)_**

In [16]:
assert (x.index == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower', 'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing', 'Main frame', 'Main shaft', 'Transformer', 'Tubular steel', 'Internals',
                      ] + list_backgroundprocesses).all(), 'The index do not correspond'
assert (x.columns == ['Output']).all(), 'The columns do not correspond'

#### **3e)** _(7.5 points)_ Calculate `d` and rename its column into "Total impacts"

In [17]:
e = S @ x
e = e.rename(columns = {'Output' : 'Stressors'})
#display(e)

d = C @ e
d = d.rename(columns = {'Stressors' : 'Total impacts'})
display(d)

,Total impacts
impact,
Acidification - AP (kg SO2-eq),5.991764e+04
Climate change - GWP100 (kg CO2-eq),4.272112e+06
Eutrophication - EP (kg PO4-eq),2.227775e+04
"Freshwater ecotoxity - FAETP 100 (kg 1,4-DCB-eq)",1.257719e+04
"Human toxicity - HTP 100 (kg 1,4-DCB-eq)",1.228823e+07
Ozone layer depletion - ODP steady state (kg CFC-11-eq),7.978973e-02
Photochemical oxidation (high NOx) - POCP (kg C2H4-eq),5.004681e+03
"Terrestrial ecotoxicity - TETP 100 (kg 1,4-DCB-eq)",1.131152e+04


**_Run the cell under to check the format of your answer(s)_**

In [18]:
assert (d.index == C.index).all(), 'The index do not correspond'
assert (d.columns == ['Total impacts'
                     ]).all(), 'The columns do not correspond'

## **4. Main contributors to GWP of the wind turbine** _(22.5 points)_

Run the cell below to get the DataFrame `Contributors_GWP` that you have to fill in by aggregating the impacts of the different sub-components of the wind turbine into its main components. This table aims to represent how much (in kgCO2eq) the turbine assembly, rotor, nacelle and tower of the wind turbine are contributing to Global warming potential 100 years. Have a look at Table 1 again to know which components belong to rotor, nacelle and tower.

In [19]:
Contributors_GWP = pd.read_csv(files_folder + 'Contributors_GWP.csv', sep = ',', index_col = [0])
display(Contributors_GWP)

,GWP100 (kgCO2eq)
Turbine assembly,NaN
Rotor,NaN
Nacelle,NaN
Tower,NaN


You have to start by performing an Advanced Contribution Analysis and calculate `Dprof`. Do not hesitate to use previous Problem Sets comprising Advanced Contribution Analysis to solve this question as you need to calculate all the steps enabling you to get `Dprof`.

In [20]:
xf = x.iloc[:15, :15]
xf_i_marr = np.diag(xf["Output"])
xf_i_m = pd.DataFrame(xf_i_marr, index = Aff.index, columns = Aff.columns)

#display(xf_i_m)

Mbf = Abf @ xf_i_m
I_Abb = np.identity(len(Abb))
Larr = np.linalg.inv(I_Abb-Abb)
L_Abb = pd.DataFrame(Larr, index = Abb.index, columns =Abb.columns )

Xbf = L_Abb @ Mbf
#display(Xbf)

Dproff = C @ Sf @ xf_i_m

Dprobf = C @ Sb @ Xbf

Dprof = Dproff + Dprobf
display(Dprof)

,Wind turbine,Turbine assembly,Rotor,Nacelle,Tower,Blade,Hub w/nose cone,Generator,Gearbox,Cover / housing,Main frame,Main shaft,Transformer,Tubular steel,Internals
impact,,,,,,,,,,,,,,,
Acidification - AP (kg SO2-eq),0.0,564.947664,0.0,0.0,0.0,404.652964,1554.201895,17470.030586,2510.792309,101.163241,1027.142308,981.491539,12620.217376,2.139880e+04,1284.205365
Climate change - GWP100 (kg CO2-eq),0.0,305033.549364,0.0,0.0,0.0,108419.356798,184143.456460,319145.382533,294196.186208,27104.839200,120352.985267,115003.963700,231676.966264,2.507354e+06,59681.447714
Eutrophication - EP (kg PO4-eq),0.0,623.558503,0.0,0.0,0.0,291.238098,1046.228696,1523.899051,1689.144941,72.809524,691.013839,660.302113,1107.535507,1.439612e+04,175.899328
"Freshwater ecotoxity - FAETP 100 (kg 1,4-DCB-eq)",0.0,229.702594,0.0,0.0,0.0,48.253146,612.967639,552.119005,996.289069,12.063286,407.572801,389.458454,403.014477,8.491100e+03,434.649304
"Human toxicity - HTP 100 (kg 1,4-DCB-eq)",0.0,156080.525636,0.0,0.0,0.0,56569.965671,611012.044322,668653.476495,992650.395081,14142.491418,406084.252533,388036.063532,487105.425938,8.460089e+06,47806.903228
Ozone layer depletion - ODP steady state (kg CFC-11-eq),0.0,0.002344,0.0,0.0,0.0,0.006676,0.003420,0.006445,0.005210,0.001669,0.002131,0.002037,0.004676,4.440259e-02,0.000780
Photochemical oxidation (high NOx) - POCP (kg C2H4-eq),0.0,58.452565,0.0,0.0,0.0,64.973902,190.948668,820.021540,307.646160,16.243476,125.855247,120.261681,593.240025,2.621984e+03,85.052981
"Terrestrial ecotoxicity - TETP 100 (kg 1,4-DCB-eq)",0.0,131.454497,0.0,0.0,0.0,37.387435,529.726695,954.718167,861.227395,9.346859,352.320298,336.661618,692.972130,7.340006e+03,65.698608


**_Run the cell under to check the format of your answer(s)_**

In [21]:
assert (Dprof.index == C.index).all(), 'The index do not correspond'
assert (Dprof.columns == ['Wind turbine', 'Turbine assembly', 'Rotor', 'Nacelle', 'Tower', 'Blade', 'Hub w/nose cone', 'Generator', 'Gearbox', 'Cover / housing', 'Main frame', 'Main shaft', 'Transformer', 'Tubular steel', 'Internals']).all(), 'The columns do not correspond'

Fill in the table Contributors_GWP by using the loc function of pandas.

In [24]:
Contributors_GWP.loc['Turbine assembly', 'GWP100 (kgCO2eq)'] = Dprof.loc['Climate change - GWP100 (kg CO2-eq)', 'Turbine assembly']
Contributors_GWP.loc['Rotor', 'GWP100 (kgCO2eq)'] = Dprof.loc['Climate change - GWP100 (kg CO2-eq)', 'Blade'] + Dprof.loc['Climate change - GWP100 (kg CO2-eq)', 'Hub w/nose cone']
Contributors_GWP.loc['Nacelle', 'GWP100 (kgCO2eq)'] = sum(Dprof.iloc[1, 7:13])
Contributors_GWP.loc['Tower', 'GWP100 (kgCO2eq)'] = sum(Dprof.iloc[1, 13:15])
display(Contributors_GWP)

,GWP100 (kgCO2eq)
Turbine assembly,3.050335e+05
Rotor,2.925628e+05
Nacelle,1.107480e+06
Tower,2.567035e+06


**_Run the cell under to check the format of your answer(s)_**

In [25]:
assert Contributors_GWP.isnull().values.any() == False, 'There are still NaN values in the matrix'
assert np.round(Contributors_GWP['GWP100 (kgCO2eq)'].sum()) == np.round(d.loc["Climate change - GWP100 (kg CO2-eq)", 'Total impacts']), 'Make sure you have not forgotten any wind turbine component'
assert (Contributors_GWP.index == ['Turbine assembly', 'Rotor', 'Nacelle', 'Tower'
                   ]).all(), 'The index do not correspond'
assert (Contributors_GWP.columns == ['GWP100 (kgCO2eq)'
                     ]).all(), 'The columns do not correspond'

Which main component is the main contributor to GWP from the wind turbine?

1. Turbine assembly
2. Rotor
3. Nacelle
4. Tower

Define your answer as the variables `answer_4a`. Your answer should be `answer_4a = {1}` if you think the "1. Turbine assembly" has the largest contribution to GWP.

In [26]:
answer_4a = {4}

**_Run the cell under to check the format of your answer(s)_**

In [27]:
assert all (type(i) in [set] for i in [answer_4a]), 'The answer(s) must be a set of values'

For the component which is having the main contribution to the total impact, which sub-component is the main contributor to GWP from the wind turbine?

1. Blade
2. Hub w/nose cone
3. Generator
4. Gearbox
5. Cover / housing
6. Main frame
7. Main shaft
8. Transformer
9. Tubular steel
10. Internals

Define your answer as the variables `answer_4b`. Your answer should be `answer_4b = {1}` if you think the "1. Blade" has the largest contribution.

In [28]:
answer_4b = {9}

**_Run the cell under to check the format of your answer(s)_**

In [29]:
assert all (type(i) in [set] for i in [answer_4b]), 'The answer(s) must be a set of values'

## **5. Structural Path Analysis** _(10 points)_

The cell below performs a Structural Path Analysis for the climate change impact category for the wind turbine. We are just collecting and displaying the 20 first paths. 

What pathways/processes contribute to the most impacts? Provide a brief quantitative analysis in the form of a paragraph that you have to **deliver on Blackboard**.

**_Run the cell under to run the Structural Path Analysis - you have nothing to code, just run the cell!_** The cell might take a few seconds to run, be patient :-)

In [30]:
### Structural Path Analysis for the windturbine
SPA_windturbine = run_SPA.run_SPA_using_pyspa(A, C, S, y, 
                            "Climate change - GWP100 (kg CO2-eq)", 'DR_GHG_emissions_(kgCO2e)', 'TR_GHG_emissions_(kgCO2e)',
                            6, 'files_PS9_V1_0/run_SPA_folder/Thresholds_template_perc.csv')

Extracting names of satellites...Done
Reading Thresholds...Done
Validating read data...The A matrix loaded is square and contains 215 sectors/processes across 1 region(s), and is described in the infosheet provided, for 1 satellite(s)...Done
Generating vectors of direct and total multipliers...Done
------ Ready to conduct the Structural Path Analysis ------
Supply Chain object created, extracting pathways, which will take some time...
Started at 13:53:14
Now calculating remainders
Ended at 13:53:23. It took 9 seconds to extract 10878 pathways and calculate 10877 remainders.
Started at 13:53:14


In [31]:
### Displaying results of Structural Path Analysis for the windturbine
display(SPA_windturbine)

,% of total intensity,direct intensity of last node in pathway,Stage 1 direct intensity,Stage 1,Stage 2 direct intensity,Stage 2,Stage 3 direct intensity,Stage 3,Stage 4 direct intensity,Stage 4,Stage 5 direct intensity,Stage 5,Stage 6 direct intensity,Stage 6
0,9.842249%,420471.8953037246,0.0,Turbine assembly,0.0,Tower,0.0,Tubular steel,420471.8953037246,Fabricated metal products; except machinery an...,,0,0,0
1,9.537384%,407447.7045419984,0.0,Turbine assembly,0.0,Tower,0.0,Tubular steel,420471.8953037246,Fabricated metal products; except machinery an...,407447.7045419984,Basic iron and steel and of ferro-alloys and f...,,0
2,5.282136%,225658.77259470962,0.0,Turbine assembly,0.0,Tower,0.0,Tubular steel,420471.8953037246,Fabricated metal products; except machinery an...,225658.77259470962,Electricity by coal - TJ,,0
3,4.435635%,189495.29912786005,0.0,Turbine assembly,189495.29912786005,Electricity by gas - TJ,,0,0,0,0,0,0,0
4,3.109851%,132856.304166396,0.0,Turbine assembly,0.0,Tower,0.0,Tubular steel,420471.8953037246,Fabricated metal products; except machinery an...,37019.869669449545,Aluminium and aluminium products - tonnes,132856.304166396,Electricity by coal - TJ
5,1.895350%,80971.4743301679,0.0,Turbine assembly,0.0,Nacelle,0.0,Generator,80971.4743301679,Copper products - tonnes,,0,0,0
6,1.888302%,80670.37485566584,0.0,Turbine assembly,0.0,Tower,0.0,Tubular steel,420471.8953037246,Fabricated metal products; except machinery an...,407447.7045419984,Basic iron and steel and of ferro-alloys and f...,80670.37485566584,Basic iron and steel and of ferro-alloys and f...
7,1.775453%,75849.34491111177,0.0,Turbine assembly,0.0,Nacelle,0.0,Generator,80971.4743301679,Copper products - tonnes,75849.34491111177,Copper ores and concentrates - tonnes,,0
8,1.367948%,58440.2814730777,0.0,Turbine assembly,0.0,Nacelle,0.0,Transformer,58440.2814730777,Copper products - tonnes,,0,0,0
9,1.281414%,54743.44024019372,0.0,Turbine assembly,0.0,Nacelle,0.0,Transformer,58440.2814730777,Copper products - tonnes,54743.44024019372,Copper ores and concentrates - tonnes,,0


## **6. Impact per kWh of electricity** _(10 points)_

Given the following information:
1. The wind turbine is a 5 MW turbine
2. The expected lifetime of the turbine is 25 years
3. The full load hours per year is 3000 hours

#### **6a)** Impact per kWh of electricity produced by the wind turbine _(5 points)_ 

What is the climate impact of 1 kWh of electricity produced by the wind turbine we have modelled? 
1. Around 11.4 gCO2eq
2. Around 6.4 gCO2eq
3. Around 8.9 gCO2eq

Define your answer as the variable `answer_6a` (e.g. `answer_6a = {1}` if you think the correct statement is statement {1} or `answer_6a = {1, 2}` if you think the correct statements are statements {1, 2}).

In [33]:
impact = d.loc['Climate change - GWP100 (kg CO2-eq)'] / (5000*3000*25)
#print(impact)
answer_6a = {1}

**_Run the cell under to check the format of your answer(s)_**

In [34]:
assert all (type(i) in [set] for i in [answer_6a]), 'The answer(s) must be a set of values'

#### **6b)** Impact per kWh of electricity produced by a wind park _(5 points)_ 

This analysis only includes the direct inputs into the manufacturing of the wind turbine, and excludes foundations, electrical connections, offshore operations with ships, spare parts, installation of the turbines, maintenance, etc, which will also make part of a wind park and its operation. Use Arvesen et al (2013) to find the total climate impacts per kWh produced in a wind park (use reference scenario). Compare the impacts with “our” wind turbine. How much of the total impact are we covering, how much are we missing?

1. Our study covers between 20 and 40 per cent of the emissions impacting global warming from 1 kWh produced in a wind park. 
2. Our study covers less than 20 per cent of the emissions impacting global warming from 1 kWhproduced in a wind park. 
3. Our study covers between 40 and 60 per cent of the emissions impacting global warming from 1 kWh produced in a wind park.  

Define your answer as the variable `answer_6b` (e.g. `answer_6b = {1}` if you think the correct statement is statement {1} or `answer_6b = {1, 2}` if you think the correct statements are statements {1, 2}).

In [35]:
answer_6b = {1}

**_Run the cell under to check the format of your answer(s)_**

In [36]:
assert all (type(i) in [set] for i in [answer_6b]), 'The answer(s) must be a set of values'